In [1]:
# !pip3 install malaya-speech -U --no-deps

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import malaya_speech
import json

malaya_speech.__version__

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

'1.2.6.1'

In [5]:
from glob import glob
malay = sorted(glob('/home/ubuntu/wav2vec2/malay-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
singlish = sorted(glob('/home/ubuntu/wav2vec2/singlish-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
mandarin = sorted(glob('/home/ubuntu/wav2vec2/mandarin-test/*.wav'), key = lambda x: int(x.split('/')[-1].replace('.wav', '')))
len(malay), len(singlish), len(mandarin)

(765, 3579, 614)

In [6]:
with open('/home/ubuntu/wav2vec2/malay-test.json') as fopen:
    malay_label = json.load(fopen)
with open('/home/ubuntu/wav2vec2/singlish-test.json') as fopen:
    singlish_label = json.load(fopen)
with open('/home/ubuntu/wav2vec2/mandarin-test.json') as fopen:
    mandarin_label = json.load(fopen)
    
len(malay_label), len(singlish_label), len(mandarin_label)

(765, 3579, 614)

In [7]:
from sklearn.utils import shuffle

audio = malay + singlish + mandarin
labels = malay_label + singlish_label + mandarin_label
audio, labels = shuffle(audio, labels)
test_set = list(zip(audio, labels))
test_set[:10]

[('/home/ubuntu/wav2vec2/singlish-test/2502.wav',
  'force to smile a forced smile'),
 ('/home/ubuntu/wav2vec2/malay-test/610.wav',
  'kenapa tidak ada masalah dengan ucapan bebas'),
 ('/home/ubuntu/wav2vec2/malay-test/425.wav',
  'aku whatsapp isteri aku yang aku nak bunuh diri'),
 ('/home/ubuntu/wav2vec2/singlish-test/2662.wav',
  'the villagers pray for plentiful harvest at the start of every year'),
 ('/home/ubuntu/wav2vec2/malay-test/247.wav',
  'agar bisa segera keluar dari ruangan maut pak dadi mulai keluar kelas'),
 ('/home/ubuntu/wav2vec2/singlish-test/3274.wav',
  'if you cant settle my request you can always proceed to hearing'),
 ('/home/ubuntu/wav2vec2/singlish-test/2757.wav',
  'and that was the start of something special'),
 ('/home/ubuntu/wav2vec2/singlish-test/1540.wav',
  'singapore has grown beyond our wildest dreams'),
 ('/home/ubuntu/wav2vec2/singlish-test/1479.wav',
  'they tried to keep it a secret but he was able to figure everything out'),
 ('/home/ubuntu/wav2v

In [8]:
import kenlm
from pyctcdecode.language_model import LanguageModel

In [9]:
lm = malaya_speech.stt.language_model(model = 'bahasa-manglish-combined')

In [10]:
kenlm_model = kenlm.Model(lm)
language_model = LanguageModel(kenlm_model, alpha = 0.01, beta = 0.5)

In [11]:
y, _ = malaya_speech.load(test_set[0][0])
y1, _ = malaya_speech.load(test_set[1][0])

In [12]:
model = malaya_speech.stt.deep_transducer(model = 'conformer-stack-3mixed')

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

2022-06-11 22:41:46.048608: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-11 22:41:46.052470: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-11 22:41:46.052488: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-11 22:41:46.052492: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-06-11 22:41:46.052556: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Not found: was unable to find libcuda.so DSO loaded into this program
2022-06-11 22:41:46.052579: I

In [13]:
%%time

model.beam_decoder_lm([y, y1], language_model)

CPU times: user 8.97 s, sys: 256 ms, total: 9.23 s
Wall time: 6.22 s


['force the small afford small',
 'kenapa tidak ada masalah dengan ucapan bebas']

In [14]:
def calculate_cer(actual, hyp):
    """
    Calculate CER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    actual = actual.replace(' ', '')
    hyp = hyp.replace(' ', '')
    return Lev.distance(actual, hyp) / len(actual)


def calculate_wer(actual, hyp):
    """
    Calculate WER using `python-Levenshtein`.
    """
    import Levenshtein as Lev

    b = set(actual.split() + hyp.split())
    word2char = dict(zip(b, range(len(b))))

    w1 = [chr(word2char[w]) for w in actual.split()]
    w2 = [chr(word2char[w]) for w in hyp.split()]

    return Lev.distance(''.join(w1), ''.join(w2)) / len(actual.split())

In [15]:
from tqdm import tqdm

wer, cer = [], []
wer_lm, cer_lm = [], []

batch_size = 4
for i in tqdm(range(0, len(audio), batch_size)):
    
    batch_x = audio[i: i + batch_size]
    batch_y = labels[i: i + batch_size]
    
    ys = [malaya_speech.load(b)[0] for b in batch_x]
    pred = model.beam_decoder(ys)
    pred_lm = model.beam_decoder_lm(ys, language_model)
    
    for k in range(len(pred)):
        
        wer.append(calculate_wer(batch_y[k], pred[k]))
        cer.append(calculate_cer(batch_y[k], pred[k]))
        
        wer_lm.append(calculate_wer(batch_y[k], pred_lm[k]))
        cer_lm.append(calculate_cer(batch_y[k], pred_lm[k]))

  5%|▍         | 61/1240 [11:39<4:01:44, 12.30s/it]2022-06-11 22:53:56.722653: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 37785600 exceeds 10% of free system memory.
2022-06-11 22:54:00.909883: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 37785600 exceeds 10% of free system memory.
  6%|▋         | 78/1240 [15:10<4:17:00, 13.27s/it]2022-06-11 22:57:27.787498: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 45711360 exceeds 10% of free system memory.
2022-06-11 22:57:32.224188: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 45711360 exceeds 10% of free system memory.
100%|██████████| 1240/1240 [3:44:53<00:00, 10.88s/it] 


In [16]:
import numpy as np

np.mean(wer), np.mean(cer), np.mean(wer_lm), np.mean(cer_lm)

(0.23476848812196416,
 0.13394483674348717,
 0.22924182213367433,
 0.13070185396400735)

In [17]:
index_malay = [no for no, i in enumerate(audio[: len(wer)]) if 'malay-test/' in i]
index_singlish = [no for no, i in enumerate(audio[: len(wer)]) if 'singlish-test/' in i]
index_mandarin = [no for no, i in enumerate(audio[: len(wer)]) if 'mandarin-test/' in i]

In [18]:
np.mean(np.array(wer)[index_malay]), np.mean(np.array(cer)[index_malay]), np.mean(np.array(wer_lm)[index_malay]), np.mean(np.array(cer_lm)[index_malay])

(0.4818578988186831,
 0.26272383033777974,
 0.4732879865232807,
 0.2588061061370562)

In [19]:
np.mean(np.array(wer)[index_singlish]), np.mean(np.array(cer)[index_singlish]), np.mean(np.array(wer_lm)[index_singlish]), np.mean(np.array(cer_lm)[index_singlish])

(0.19609774671596963,
 0.11304379568019195,
 0.19033112424535406,
 0.10952327667125457)

In [20]:
np.mean(np.array(wer)[index_mandarin]), np.mean(np.array(cer)[index_mandarin]), np.mean(np.array(wer_lm)[index_mandarin]), np.mean(np.array(cer_lm)[index_mandarin])

(0.1523241628924268,
 0.09532740330032738,
 0.15198786771062786,
 0.09454285594833946)